In [1]:
# This notebook is  used to evaluate perofrmance of our RAG ingestion and query pipeline.
# Method:
#   - Generated test data using a sample GitHub repository (offline process not in this notebook)
#     - https://docs.ragas.io/en/stable/getstarted/rag_testset_generation/
#   - Use our ingestion pipeline to parse the same reporisiroty and index in our vector store
#   - Use RAGAS on teadt dataset + our answers fromn our RAG and visualise the mterics.
import os
import sys
import pandas as pd

sys.path.insert(1, '/home/jovyan/work/code')
from config import VectorDBConfig, EmbeddingConfig, ProcessingConfig, ChatConfig
from config_helper import ConfigHelper
from pipeline import DocumentPipeline
from localrag import LocalRAG

from TraceSetup import get_tracer, get_logger
logger = get_logger()
tracer = get_tracer()

config_helper = ConfigHelper(False)

In [2]:
# Ingest a GitHub repository and import into our vector store

# Initialize the pipeline
pipeline = DocumentPipeline(
    vector_db_config=config_helper.vector_db_config,
    embedding_config=config_helper.embedding_config,
    logger=logger,
    tracer=tracer)

local_ingestion_file="dotnet-docs-aspire.txt"
repository="https://github.com/dotnet/docs-aspire"
with tracer.start_as_current_span(f"Starting ingesting {repository} using local ingestion file {local_ingestion_file}"):
    pipeline.process_single_file(local_ingestion_file,repository)


/home/jovyan/work/code/pipeline.py:38: UserWarning: Api key is used with an insecure connection.
  self.qdrant = QdrantClient(url=vector_db_config.url, api_key=vector_db_config.api_key)
INFO:TraceSetup:collection: mistralmxbai-embed-large vector size: 1024
INFO:TraceSetup:Collection mistralmxbai-embed-large does not exist. Will create now. Vector size is 1024
INFO:TraceSetup:Created collection: mistralmxbai-embed-large
INFO:TraceSetup:Starting repository chunking...
INFO:TraceSetup:Created chunking strategies
INFO:TraceSetup:Split content into 181 files
INFO:TraceSetup:Total chunks created: 3342


Total parts found: 363
Total files processed: 181
Sample file paths:
 1. README.md
 2. CODE_OF_CONDUCT.md
 3. SECURITY.md
Added 50 chunks for file docs/reference/aspire-faq.yml


INFO:TraceSetup:done processing the repository.


done processing the repository.


In [3]:
# Test our RAG Solution

def demonstrate_local_rag(rag):
    """Demonstrate how to use the LocalRAG class."""    
    # Example questions to test
    questions = [
        "If I use Docker Compose, why should I care about .Net Aspire?",
        "Is .Net Aspire an alternative to Kubernetes?"
    ]
    with tracer.start_as_current_span("Entering questions loop."):
        for question in questions:
            print(f"Question: {question}")
            # print("\nRelevant chunks:")
            # with tracer.start_as_current_span("rag get chunks"):
            #     chunks = rag.get_relevant_chunks(question, k=5)
            #     for i, chunk in enumerate(chunks, 1):
            #         print(chunk)
            #         break
#                    print(f"\nChunk {i}:")
#                    print(f"Source: {chunk.metadata.get('file_path', 'Unknown')}")
#                    print(f"Content: {chunk.page_content[:200]}...")
                    
            print("\nGenerated Answer:")
            with tracer.start_as_current_span("Retrieve answers."):
                answer = rag.retrieve_and_answer(question, k=10)
                print(answer)
                print("\n" + "="*80 + "\n")

rag = LocalRAG(
    vector_db_config=config_helper.vector_db_config,
    embedding_config=config_helper.embedding_config, 
    chat_config=config_helper.chat_config,
    logger=logger,
    tracer=tracer)

with tracer.start_as_current_span("Starting demo"):
    demonstrate_local_rag(rag)

Question: If I use Docker Compose, why should I care about .Net Aspire?

Generated Answer:


/home/jovyan/work/code/localrag.py:43: UserWarning: Api key is used with an insecure connection.
  self.qdrant = QdrantClient(url=vector_db_config.url, api_key=vector_db_config.api_key)


 .NET Aspire is designed to simplify the process of running and managing multi-container applications without the need for learning additional tools like Docker Compose. Unlike Docker Compose which requires YAML configuration, .NET Aspire uses declarative code written in a real programming language (C#), making it easier to reason about and debug your application.

   In addition, .NET Aspire projects are designed to run in containers and can use any OCI-compliant container runtime like Docker Desktop or Podman. This allows for better compatibility with various development environments. Furthermore, .NET Aspire provides a better developer experience by offering features like IntelliSense and environment variable replacements directly within your IDE.

   However, it's important to note that if you are already comfortable using Docker Compose and prefer its workflow, there is no requirement to switch to .NET Aspire. The choice between the two ultimately depends on your personal preferen